In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-246179
Azure region: eastus2
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-246179


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "compClusToriLab1"
nodes = 4
vm_sz = "Standard_D2_V2"
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_sz,
                                                            max_nodes=nodes)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
#from train.py     parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization"
#                  parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")

ps = RandomParameterSampling( {
        "--C": uniform(0.05, 0.5),
        "--max_iter": choice(25, 50, 75, 100)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 1, slack_factor = 0.15,delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            #arguments=['--arg1', arg1_val, '--arg2', arg2_val],
                            compute_target=cluster_name,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name= 'Accuracy',
                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=50)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run = exp.submit(hyperdrive_config)
run.get_details()
run.wait_for_completion(show_output=True)

RunId: HD_37248d36-edfa-42ab-bc1c-2a191fbc8290
Web View: https://ml.azure.com/runs/HD_37248d36-edfa-42ab-bc1c-2a191fbc8290?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-246179/workspaces/quick-starts-ws-246179&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-12-20T21:58:21.951653][GENERATOR][INFO]Trying to sample '50' jobs from the hyperparameter space
[2023-12-20T21:58:22.6331843Z][SCHEDULER][INFO]Scheduling job, id='HD_37248d36-edfa-42ab-bc1c-2a191fbc8290_0' 
[2023-12-20T21:58:22.7517180Z][SCHEDULER][INFO]Scheduling job, id='HD_37248d36-edfa-42ab-bc1c-2a191fbc8290_1' 
[2023-12-20T21:58:22.9001346Z][SCHEDULER][INFO]Scheduling job, id='HD_37248d36-edfa-42ab-bc1c-2a191fbc8290_2' 
[2023-12-20T21:58:22.9702227Z][SCHEDULER][INFO]Scheduling job, id='HD_37248d36-edfa-42ab-bc1c-2a191fbc8290_3' 
[2023-12-20T21:58:23.1948766Z][SCHEDULER][INFO]Scheduling job, id='HD_37248d36-edfa-42ab-bc1c-2a191fbc8290_4' 
[2023-12-2

{'runId': 'HD_37248d36-edfa-42ab-bc1c-2a191fbc8290',
 'target': 'compClusToriLab1',
 'status': 'Completed',
 'startTimeUtc': '2023-12-20T21:58:21.131665Z',
 'endTimeUtc': '2023-12-20T22:25:02.293032Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '78c473df-a7bd-4762-972e-4e19e60f1dc6',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_37248d36-edfa-42ab-bc1c-2a191fbc8290_31',
  'score': '0.9091047040971167',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_37248d36-edfa-42ab-bc1c-2a191fbc8290_31'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'teleme

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()

if best_run == None:
    print('No best run!')
else:
    metrics = best_run.get_metrics()
    best_run_details = best_run.get_details()
    print(metrics)


{'Regularization Strength:': 0.11684859776738893, 'Max iterations:': 75, 'Accuracy': 0.9091047040971169}


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path = data_path)

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###)
training_data = x.copy()
training_data['y'] = y
#print(x.head(5))

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_data,
    label_column_name='y',
    validation_size=0.2, #this was commmented
    n_cross_validations=5)

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
run2 = exp.submit(automl_config)

2023-12-20:22:27:28,121 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_2bb59b1b-6421-4ea5-a0ed-e64fd3132951,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/12/20 22:41:07 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-12-20:23:00:10,526 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-12-20:23:00:12,741 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-12-20:23:00:12,742 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-12-20:23:00:14,740 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-12-20:

In [10]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
results2 = run2.get_best_child()
metrics2 = results2.get_metrics()
best_run2 = results2.get_details().get('runID')

print('The best run was: ')
print(results2)
print('The best run reported the following metrics: ')
print(metrics2)

The best run was: 
Run(Experiment: udacity-project,
Id: AutoML_2bb59b1b-6421-4ea5-a0ed-e64fd3132951_20,
Type: None,
Status: Completed)
The best run reported the following metrics: 
{'log_loss': 0.19273476780153664, 'precision_score_weighted': 0.9171627450653252, 'norm_macro_recall': 0.5660860545737382, 'AUC_macro': 0.9489319942430384, 'recall_score_micro': 0.9188467374810318, 'f1_score_macro': 0.7893094722674932, 'accuracy': 0.9188467374810318, 'recall_score_macro': 0.7830430272868691, 'recall_score_weighted': 0.9188467374810318, 'precision_score_micro': 0.9188467374810318, 'weighted_accuracy': 0.9521162280102653, 'average_precision_score_macro': 0.826701863795521, 'precision_score_macro': 0.7961465894132969, 'AUC_micro': 0.9813146419023628, 'f1_score_micro': 0.9188467374810318, 'f1_score_weighted': 0.9179335014901182, 'average_precision_score_micro': 0.9820947363489815, 'average_precision_score_weighted': 0.9565449951735934, 'matthews_correlation': 0.5789871731609386, 'AUC_weighted': 

In [13]:
#clean up
cpu_cluster.delete()